# Introduction

This project focuses on augmenting and complementing data skills through common software engineering tasks. These tasks are designed to enhance one's proficiency in the realm of data science and make them a more attractive job candidate to potential employers. The core objective is to develop and deploy a publicly accessible web application on a cloud service.

In the realm of data analysis, this Jupyter notebook serves as the preliminary step, concentrating on Exploratory Data Analysis (EDA) of a dataset containing car sales advertisements. The analysis conducted here lays the groundwork for the subsequent development of a web application using Streamlit.

**Note**: The visualizations in this notebook are powered by Plotly, which might not render directly on GitHub. For an optimal viewing experience, it is recommended to access the notebook through [nbviewer](https://nbviewer.org/github/tora-ds/Project_4/blob/main/EDA.ipynb). For those interested in experiencing the web application live, it is hosted on [Render](https://us-vehicle-market-dashboard.onrender.com/).

# Set Up

For this analysis, several Python libraries were utilized to facilitate data manipulation, analysis, and visualization. Here's a brief overview of the libraries and their primary functions:

- `numpy`: A fundamental package for scientific computing in Python, `numpy` provides support for working with arrays (including mathematical functions to operate on these arrays).
- `pandas`: An essential library for data manipulation and analysis, `pandas` offers data structures and functions necessary to efficiently manipulate large datasets.
- `plotly.express`: A concise interface to the `plotly` library, `plotly.express` allows for the rapid creation of a variety of visualizations with a simple and consistent API.

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

from plotly.subplots import make_subplots

# Data Preprocessing

The dataset for our analysis is sourced from the `vehicles_us.csv` file. Before delving into the exploratory data analysis (EDA), it was imperative to preprocess and tidy the data to ensure a smooth and effective analysis. Initially, any entries with missing values in the `model_year` column were eliminated.

A noticeable aspect of the dataset was that the `date_posted` column, which inherently represents date values, was formatted as a string. To rectify this and facilitate any date-related operations, this column was converted to the `datetime` format.

Furthermore, to enhance the granularity of our data and enable manufacturer-based filtering and aggregation in subsequent analyses, a new `manufacturer` column was introduced. This column was derived by isolating the first word from the existing `model` column.

In [3]:
df = pd.read_csv("vehicles_us.csv")
df = df.dropna(subset="model_year").reset_index(drop=True)
df["date_posted"] = pd.to_datetime(df["date_posted"])
df["manufacturer"] = df["model"].apply(lambda x: x.split()[0])
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
1,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
2,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford
3,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler
4,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler


While analyzing the entire dataset offers a broad overview, it's often not prudent to combine all manufacturers in a single analysis. Different car manufacturers come with distinct characteristics, market strategies, and consumer perceptions. Therefore, aggregating all manufacturers might obfuscate unique insights related to individual brands.

Recognizing this, our eventual aim for the web application is to incorporate a manufacturer filter, allowing users to select and analyze data specific to a particular car manufacturer. This way, users can derive more targeted and meaningful insights.

For the purpose of this Jupyter notebook and to maintain simplicity, we've decided to narrow our focus and analyze only the data related to BMW. By doing so, we aim to delve deeper into the characteristics and trends associated with this specific brand.

In [4]:
df = df[df["manufacturer"] == "bmw"]
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
56,3200,2004.0,bmw x5,excellent,6.0,gas,NaN,automatic,SUV,black,1.0,2019-04-09,10,bmw
81,8499,2009.0,bmw x5,good,NaN,gas,104482.0,automatic,SUV,blue,1.0,2019-03-04,52,bmw
187,22914,2015.0,bmw x5,excellent,6.0,gas,69686.0,automatic,SUV,white,1.0,2018-08-11,19,bmw
252,22914,2015.0,bmw x5,excellent,6.0,gas,69686.0,automatic,SUV,white,1.0,2018-07-07,9,bmw


To ensure a cleaner analysis, we've removed extreme values from both the `price` and `odometer` columns. We used the Interquartile Range (IQR) method, which calculates a range based on the data's quartiles. Values outside this range are considered outliers.

In [5]:
def calculate_bounds(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

price_lower, price_upper = calculate_bounds(df["price"])
df = df[(df["price"] >= price_lower) & (df["price"] <= price_upper)]

odometer_lower, odometer_upper = calculate_bounds(df["odometer"])
df = df[(df["odometer"] >= odometer_lower) & (df["odometer"] <= odometer_upper)]

# Exploratory Data Analysis

To begin our exploratory data analysis, we're interested in understanding the distribution of various categories within our dataset. To visualize these distributions, pie charts serve as a concise and effective tool, offering a snapshot of the relative proportions of each category. In our dataset, we have several categorical columns like `model`, `condition`, `fuel`, `transmission`, and `type`. For each of these columns, a pie chart provides a clear breakdown of the percentage representation of each category.

In [6]:
def create_pie_trace(df, column_name):
    fig = px.pie(df, names=column_name, hole=0.4)
    pie_data = fig.data[0]
    pie_data.update(
        textposition="inside",
        hoverinfo="label+percent+value",
        hovertemplate="%{label}<br>%{value}"
    )
    return pie_data

columns_for_pie = ["model", "condition", "fuel", "transmission", "type"]
subplot_titles = [f"Distribution of {col_name.capitalize()}" for col_name in columns_for_pie]

fig = make_subplots(rows=1, cols=5, subplot_titles=subplot_titles, specs=[[{'type':'domain'}] * 5])

for i, col_name in enumerate(columns_for_pie):
    pie_trace = create_pie_trace(df, col_name)
    fig.add_trace(pie_trace, row=1, col=i+1)

fig.update_layout(margin=dict(b=0, l=0, r=0), showlegend=False)
fig.show()

One of the most pivotal aspects of any car sales dataset is the distribution of prices. Understanding this distribution is fundamental, as it provides insights into the pricing trends and the most common price ranges for the vehicles. To visualize the distribution of prices for BMW vehicles, we've employed a histogram. A histogram offers a clear perspective on the frequency of various price ranges, allowing us to discern the predominant pricing brackets. From the resulting visualization, it's evident that the majority of BMW cars in our dataset are priced between $8,000 to $9,900.

While this histogram provides a broad overview, the richness of our dataset allows for more granular analyses. For instance, we can further break down the price distribution by factors like `model`, `condition`, `fuel`, `transmission`, and `type`. These detailed breakdowns, offering insights into how prices vary across different categories, will be available in our web application for users to explore.

In [7]:
fig = px.histogram(df, x="price", title="Distribution of Prices")
fig.update_layout(
    xaxis_title=None,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(b=0, l=0, r=0),
)
fig.show()

Another vital aspect to explore is the relationship between the price of a car and its odometer reading. Intuitively, one would expect a car with higher mileage (or a longer odometer reading) to be priced lower due to wear and tear. To visualize this relationship, we've plotted a scatter plot with the odometer readings on the x-axis and the corresponding prices on the y-axis.

In [8]:
fig = px.scatter(df, x="odometer", y="price", title="Price vs. Odometer")
fig.update_layout(
    xaxis_title="Odometer",
    yaxis_title="Price",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(b=0, l=0, r=0),
    legend_orientation="h",
)
fig.update_traces(marker=dict(size=10))
fig.show()

An interesting dimension to explore is how long vehicles, in this case, BMWs, remain listed over time. This can provide insights into the market dynamics, indicating periods of high demand or potential saturation. To visualize this trend, we've plotted a line chart that showcases the total number of days vehicles were listed for each day. This aggregates the listing durations daily, allowing us to observe any notable patterns or trends.

From the resulting chart, we can derive insights into specific periods where vehicles were listed for shorter or longer durations, potentially indicating fluctuations in demand or supply in the used BMW market.

In [9]:
df_trend = df.set_index("date_posted").resample("D").sum()

fig = px.line(df_trend, x=df_trend.index.strftime('%Y-%m-%d'), y="days_listed", title="Total Days Vehicles Were Listed Each Day")
fig.update_layout(
    xaxis_title=None,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(b=0, l=0, r=0),
)
fig.show()

# Next Step

The analysis presented in this notebook serves as a foundational overview of the dataset, focusing on specific aspects related to BMW vehicles. However, the true potential of this dataset lies in its implementation within an interactive environment.

Our next step is to bring this analysis to life on a Streamlit web application, which will be hosted on Render. In the web application, users will have the flexibility to delve deeper into the data, not just for BMW but for all manufacturers. The app will feature interactive filters, allowing users to tailor their exploration based on specific manufacturers, models, and other criteria.